In [1]:
# Jupyter relative import (else it interupts the module)
import os, sys
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
    
from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster
from cassandra.cqlengine import connection, management, query
from cassandra.cqlengine.connection import log as cql_logger
from config import get_settings
from db import create_session
import pandas as pd


session = create_session()

In [2]:
# Create test_keyspace
session.execute("""
CREATE KEYSPACE IF NOT EXISTS test_keyspace
WITH replication = { 'class': 'SimpleStrategy', 'replication_factor': 1 } 
AND durable_writes = 'true';
""")

for row in (session.execute("describe keyspaces;")):
    print("Keyspace: {}".format(row[0]))

Keyspace: system
Keyspace: system_auth
Keyspace: system_distributed
Keyspace: system_schema
Keyspace: system_traces
Keyspace: system_views
Keyspace: system_virtual_schema
Keyspace: test_keyspace


In [3]:
# Drop test_keyspace
session.execute("""DROP KEYSPACE IF EXISTS test_keyspace;""")

for row in (session.execute("describe keyspaces;")):
    print("Keyspace: {}".format(row[0]))

Keyspace: system
Keyspace: system_auth
Keyspace: system_distributed
Keyspace: system_schema
Keyspace: system_traces
Keyspace: system_views
Keyspace: system_virtual_schema


In [4]:
# Create a keyspace
session.execute("""
CREATE KEYSPACE IF NOT EXISTS cql_keyspace
WITH replication = { 'class': 'SimpleStrategy', 'replication_factor': 1 } 
AND durable_writes = 'true';
""")

# Create tables in cql_keyspace for car makes
session.execute("CREATE TABLE IF NOT EXISTS cql_keyspace.cars_by_id (id int PRIMARY KEY, make text, model text)")

# Insert a record into each keyspace table
session.execute("INSERT INTO cql_keyspace.cars_by_id (id, make, model) VALUES (1, 'Toyota', 'Camry')")

# Execute a CQL query to retrieve the record from the cars_by_id table
result = session.execute("SELECT * FROM cql_keyspace.cars_by_id WHERE id = 1")

# Convert the result to a Pandas DataFrame
pd.DataFrame(list(result)).head()


,id,make,model
0,1,Toyota,Camry


In [5]:
# Create tables in cql_keyspace for employees
session.execute("CREATE TABLE IF NOT EXISTS cql_keyspace.employees_by_id (id int PRIMARY KEY, firstname text, surname text, age int)")

# Insert a record into each keyspace table
session.execute("INSERT INTO cql_keyspace.employees_by_id (id, firstname, surname, age) VALUES (1, 'John', 'Doe', 26)")

# Execute a CQL query to retrieve the record from the cars_by_id table
result = session.execute("SELECT * FROM cql_keyspace.employees_by_id WHERE id = 1")

# Convert the result to a Pandas DataFrame
pd.DataFrame(list(result)).head()

,id,age,firstname,surname
0,1,26,John,Doe
